In [2]:
# %pip install llama-index-llms-openai
# %pip install llama-index-readers-file pymupdf

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
from pathlib import Path

# Create data directory safely (Windows / Linux / macOS)
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
import chromadb

# Setup
embed_model = AzureOpenAIEmbedding(model="text-embedding-3-large")
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection("rag_hier")
vector_store = ChromaVectorStore(chroma_collection=collection)

# Load multiple PDFs
reader = SimpleDirectoryReader("./pdfs_dir")
docs = reader.load_data()

# Hierarchical parse
node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[2048, 512, 128])
nodes = node_parser.get_nodes_from_documents(docs)
leaf_nodes = get_leaf_nodes(nodes)  # Import from llama_index.core.node_parser.relational.hierarchical

# Optional: Add parent summary (cost-optimized loop)
for leaf in leaf_nodes:
    if leaf.relationships.PARENT:
        parent = next(n for n in nodes if n.node_id == leaf.relationships.PARENT.node_id)
        summary = gpt_4o_mini_client.chat.completions.create(prompt=f"Summarize: {parent.text}")  # Truncated
        leaf.text = f"Parent hierarchy: {summary.choices[0].message.content}\n\n{leaf.text}"

# Embed and store leaves
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(leaf_nodes, embed_model=embed_model, storage_context=storage_context)
